In [1]:
import torch

In [2]:
torch.autograd.set_detect_anomaly(True)

In [7]:
action_slc_n_epochs = 100
action_slc_lr = 0.1
n_target = 2
mvt_amplitude = 3
max_coord = torch.tensor([300., 600.])
n_rollout = 2
n_step_per_rollout = 2
user_sigma = 5
decay_factor = 0.9

x = torch.tensor([[0.25, 0.25], [0.75, 0.75]]) * max_coord
b = torch.ones(n_target)

a = torch.nn.Parameter(torch.zeros(n_target))

a_opt = torch.optim.Adam([a, ], lr=action_slc_lr)

for step in range(action_slc_n_epochs):

    old_a = a.clone()
    a_opt.zero_grad()

    first_action = torch.sigmoid(a)*360
    total_efe = 0
    for _ in range(n_rollout):
        
        efe_rollout = 0
        
        # Sample the user goal --------------------------
        
        q = torch.softmax(b, dim=0)
        goal = torch.multinomial(q, 1)[0]
        
        # -----------------------------------------------

        x_rol = x.clone()
        b_rol = b.clone()

        action_plan = torch.zeros((n_step_per_rollout, n_target))
        action_plan[0] = first_action
        if n_step_per_rollout > 1:
            action_plan[1:] = torch.rand((n_step_per_rollout-1, n_target)) * 360

        for step in range(n_step_per_rollout):

            action = action_plan[step]

            # ---- Update positions based on action ---------------------------------------------

            x_rol_prev = x_rol.clone()

            for i in range(n_target):
                angle = action[i]
                if 90 < angle <= 270:
                    x_prime = -1
                else:
                    x_prime = 1
                y_prime = torch.tan(torch.deg2rad(angle)) * x_prime

                norm = mvt_amplitude / torch.sqrt(y_prime**2 + x_prime**2)
                movement = torch.tensor([x_prime, y_prime]) * norm
                x_rol[i] += movement

            for coord in range(2):
                for target in range(n_target):
                    if x_rol[target, coord] > max_coord[coord]:
                        x_rol[target, coord] = max_coord[coord]

            # ------------------------------------------------------------------------------            
            # Evaluate epistemic value -----------------------------------------------------
            # ------------------------------------------------------------------------------

            # Simulate action based on goal ----------------------------------------

            delta = x_rol[goal] - x_rol_prev[goal]
            noise = torch.randn(2) * user_sigma
            y = delta + noise

            # Compute log probability of user action given a specific goal in mind -------
            logp_y = torch.zeros(n_target)
            for target in range(n_target):
                for coord in range(2):
                    d = x_rol[target, coord] - x_rol_prev[target, coord]
                    logp_coord = torch.distributions.Normal(d, user_sigma).log_prob(y[coord])
                    logp_y[target] += logp_coord
            
            logq = torch.log_softmax(b_rol - b_rol.max(), dim=0)
            logp_yq = logq + logp_y

            # Revise belief -------------------

            belief_updt_n_epochs = 100
            belief_updt_lr = 0.1

            b_rol = torch.nn.Parameter(b_rol)
            b_opt = torch.optim.Adam([b_rol, ], lr=belief_updt_lr)

            for step in range(belief_updt_n_epochs):

                old_b = b_rol.clone()

                b_opt.zero_grad()
                q_rol = torch.softmax(b_rol - b_rol.detach().max(), dim=0)
                kl_div = torch.sum(q_rol * (q_rol.log() - logp_yq))
                kl_div = q_rol.sum()
                kl_div.backward(retain_graph=True)
                b_opt.step()

                if torch.isclose(old_b, b_rol).all():
                    break

            # -----------------------------------
            
            epistemic_value = kl_div

            efe_step = - epistemic_value  # Need to add intrinsic value

            efe_rollout += decay_factor**step * efe_step

        total_efe += efe_rollout

    total_efe /= n_rollout
    total_efe.backward()
    a_opt.step()

    if torch.isclose(old_a, a).all():
        break
